In [52]:
import pandas as pd
import plotly.express as px
from google.oauth2 import service_account
from google.cloud import bigquery
import streamlit as st
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from pingouin import mixed_anova, read_dataset, pairwise_ttests
#import pingouin as pg


# En este mundo de las IDEs hay que actualizarse
Esto es una prueba de markdown en pycharm. Por cierto, la ide es una pasada.

In [91]:
# Create API client.
credentials = service_account.Credentials.from_service_account_info(
    st.secrets["gcp_service_account"]
)
client = bigquery.Client(credentials=credentials)

# Perform query.
# Uses st.cache to only rerun when the query changes or after 10 min.
#@st.cache(ttl=600)
def run_query():
    df_1 = client.query("select * from psycho_research.df_scales").to_dataframe()
    df_2 = client.query("select * from psycho_research.df_demographics").to_dataframe()
    df = pd.merge(df_1, df_2, how = 'left', left_on = ['email', 'Generation_key'], right_on = ['email', 'Generation_key'])
    #me quedo sólo con los completos:
    df = df.loc[df.Completion_prepost == 'Completed']
    #añado un id
    df['id_subject'] = df.email + df.Generation_key
    df = df.loc[~df.Teacher.isin(["Carola", "Beatriz"])]
    return df

df = run_query()
df

,id_x,email,Form,Generation_key,Completion_prepost,DASS_depression,DASS_anxiety,DASS_stress,DASS_global,FFMQ_act,...,Meditation_interrupted_months,Meditation_frequency_weekly,Meditation_totaltime_daily,Informal_practice_totaltime_daily,Meditation_last_time_hours,Meditation_retire,Meditation_retire_amount,Meditation_retire_totaldays,Meditation_retire_last_time_months,id_subject
0,77,ninalopezfaz@hotmail.com,Formulario Pre,MBSR_2017_Agustin_2_X_t_p,Completed,2.0,14.0,18.0,34.0,13.0,...,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,ninalopezfaz@hotmail.comMBSR_2017_Agustin_2_X_t_p
1,79,algomasquenutricion@gmail.com,Formulario Pre,MBSR_2017_Gustavo_2_M_m_p,Completed,12.0,12.0,34.0,58.0,11.0,...,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,algomasquenutricion@gmail.comMBSR_2017_Gustavo...
2,82,RVR.ROCIO@GMAIL.COM,Formulario Pre,MBSR_2017_Agustin_2_X_m_p,Completed,16.0,18.0,20.0,54.0,10.0,...,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,RVR.ROCIO@GMAIL.COMMBSR_2017_Agustin_2_X_m_p
3,83,monicajigo@hotmail.com,Formulario Pre,MBSR_2017_Gustavo_2_M_t_p,Completed,16.0,14.0,30.0,60.0,12.0,...,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,monicajigo@hotmail.comMBSR_2017_Gustavo_2_M_t_p
4,84,sarasgpsicologia@gmail.com,Formulario Pre,MBSR_2017_Agustin_2_X_m_p,Completed,8.0,2.0,10.0,20.0,13.0,...,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,sarasgpsicologia@gmail.comMBSR_2017_Agustin_2_...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1584,900,lauramestre@gmail.com,Formulario Post,MBSR_2019_Agustin_2_M_t_p,Completed,0.0,0.0,4.0,4.0,18.0,...,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,lauramestre@gmail.comMBSR_2019_Agustin_2_M_t_p
1585,1122,palomagilardi@gmail.com,Formulario Pre,CCT_2021_Silvia_2_X_m_o,Completed,12.0,16.0,20.0,48.0,9.0,...,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,palomagilardi@gmail.comCCT_2021_Silvia_2_X_m_o
1586,1083,martadoble@gmail.com,Formulario Pre,MBSR_2021_Agustin_1_X_t_o,Completed,16.0,10.0,24.0,50.0,8.0,...,12.0,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,martadoble@gmail.comMBSR_2021_Agustin_1_X_t_o
1587,1061,morenoisa@telefonica.net,Formulario Pre,MBSR_2018_Gustavo_2_M_t_p,Completed,2.0,4.0,16.0,22.0,17.0,...,36.0,1.0,30.0,0.0,48.0,Sí,1.0,2.0,120.0,morenoisa@telefonica.netMBSR_2018_Gustavo_2_M_t_p


In [94]:

posibles_comparaciones = ['Format', 'Teacher', 'Gender']
lista_escalas = ['DASS_stress', 'DASS_anxiety', 'DASS_depression']
option = 'Format'
for escala in lista_escalas:
    height = len(df[option].unique().tolist()) * 400
    fig = px.box(df, x="Program", y=escala, points='all',
             color = "Form" , title=escala + ' by ' + option, notched=True,
             facet_row = option,
             color_discrete_sequence=px.colors.qualitative.Pastel, template="plotly_dark",
             height= height
            )
    aov = pg.anova(dv=escala, between=["Program", option], data=df,
               detailed=True)
    anova_mixed = df.mixed_anova(dv=escala, between=option, within='Form', subject='id_subject', effsize="ng2", correction = True ).round(3)
    posthocs = pg.pairwise_ttests(dv=escala, within='Form', subject='id_subject', between=option, parametric=False, effsize = 'cohen', interaction = True, correction = True, data = df ).round(3)
    print(aov,anova_mixed, posthocs)
    fig.show()


/Users/gustavodiez/Library/Caches/pypoetry/virtualenvs/streamlit-dash-9GqNcUWI-py3.9/lib/python3.9/site-packages/statsmodels/compat/pandas.py:65: FutureWarning:

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.



             Source            SS      DF          MS         F     p-unc  \
0           Program    160.433540     1.0  160.433540  2.786909  0.095302   
1            Format     10.186529     1.0   10.186529  0.176951  0.674083   
2  Program * Format     45.202856     1.0   45.202856  0.785224  0.375728   
3          Residual  68159.146510  1184.0   57.566847       NaN       NaN   

        np2  
0  0.002348  
1  0.000149  
2  0.000663  
3       NaN           Source        SS  DF1  DF2        MS        F  p-unc    ng2  eps
0       Format    24.187    1  592    24.187    0.310  0.578  0.000  NaN
1         Form  5027.892    1  592  5027.892  173.891  0.000  0.074  1.0
2  Interaction    38.949    1  592    38.949    1.347  0.246  0.001  NaN         Contrast             Form                A               B Paired  \
0           Form                -  Formulario Post  Formulario Pre   True   
1         Format                -                o               p  False   
2  Form * Format  For

/Users/gustavodiez/Library/Caches/pypoetry/virtualenvs/streamlit-dash-9GqNcUWI-py3.9/lib/python3.9/site-packages/pingouin/parametric.py:1519: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Users/gustavodiez/Library/Caches/pypoetry/virtualenvs/streamlit-dash-9GqNcUWI-py3.9/lib/python3.9/site-packages/pingouin/pairwise.py:428: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Users/gustavodiez/Library/Caches/pypoetry/virtualenvs/streamlit-dash-9GqNcUWI-py3.9/lib/python3.9/site-packages/pingouin/pairwise.py:428: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Users/gustavodiez/Library/Caches/pypoetry/virtualenvs/streamlit-dash-9GqNcUWI-py3.9/lib/python3.9/site-packages/pingouin/pairwise.py:461: FutureWarning:

The frame.append met

             Source            SS      DF          MS         F     p-unc  \
0           Program    114.570984     1.0  114.570984  3.299152  0.069568   
1            Format     14.745967     1.0   14.745967  0.424620  0.514767   
2  Program * Format      7.868889     1.0    7.868889  0.226590  0.634152   
3          Residual  41117.251676  1184.0   34.727409       NaN       NaN   

        np2  
0  0.002779  
1  0.000359  
2  0.000191  
3       NaN           Source       SS  DF1  DF2       MS       F  p-unc    ng2  eps
0       Format   28.187    1  592   28.187   0.557  0.456  0.001  NaN
1         Form  750.114    1  592  750.114  42.745  0.000  0.018  1.0
2  Interaction  133.121    1  592  133.121   7.586  0.006  0.003  NaN         Contrast             Form                A               B Paired  \
0           Form                -  Formulario Post  Formulario Pre   True   
1         Format                -                o               p  False   
2  Form * Format  Formulario Post

/Users/gustavodiez/Library/Caches/pypoetry/virtualenvs/streamlit-dash-9GqNcUWI-py3.9/lib/python3.9/site-packages/pingouin/parametric.py:1519: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Users/gustavodiez/Library/Caches/pypoetry/virtualenvs/streamlit-dash-9GqNcUWI-py3.9/lib/python3.9/site-packages/pingouin/pairwise.py:428: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Users/gustavodiez/Library/Caches/pypoetry/virtualenvs/streamlit-dash-9GqNcUWI-py3.9/lib/python3.9/site-packages/pingouin/pairwise.py:428: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Users/gustavodiez/Library/Caches/pypoetry/virtualenvs/streamlit-dash-9GqNcUWI-py3.9/lib/python3.9/site-packages/pingouin/pairwise.py:461: FutureWarning:

The frame.append met

             Source            SS      DF         MS         F     p-unc  \
0           Program      6.200718     1.0   6.200718  0.115546  0.733977   
1            Format      2.183677     1.0   2.183677  0.040691  0.840170   
2  Program * Format     28.108626     1.0  28.108626  0.523785  0.469375   
3          Residual  63538.721291  1184.0  53.664461       NaN       NaN   

        np2  
0  0.000098  
1  0.000034  
2  0.000442  
3       NaN           Source        SS  DF1  DF2        MS        F  p-unc    ng2  eps
0       Format     3.330    1  592     3.330    0.042  0.838  0.000  NaN
1         Form  2968.758    1  592  2968.758  129.175  0.000  0.047  1.0
2  Interaction    35.608    1  592    35.608    1.549  0.214  0.001  NaN         Contrast             Form                A               B Paired  \
0           Form                -  Formulario Post  Formulario Pre   True   
1         Format                -                o               p  False   
2  Form * Format  Formular

/Users/gustavodiez/Library/Caches/pypoetry/virtualenvs/streamlit-dash-9GqNcUWI-py3.9/lib/python3.9/site-packages/pingouin/parametric.py:1519: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Users/gustavodiez/Library/Caches/pypoetry/virtualenvs/streamlit-dash-9GqNcUWI-py3.9/lib/python3.9/site-packages/pingouin/pairwise.py:428: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Users/gustavodiez/Library/Caches/pypoetry/virtualenvs/streamlit-dash-9GqNcUWI-py3.9/lib/python3.9/site-packages/pingouin/pairwise.py:428: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Users/gustavodiez/Library/Caches/pypoetry/virtualenvs/streamlit-dash-9GqNcUWI-py3.9/lib/python3.9/site-packages/pingouin/pairwise.py:461: FutureWarning:

The frame.append met

In [90]:
anova

,Source,SS,DF1,DF2,MS,F,p-unc,ng2,eps
0,Format,1.005,1,608,1.005,0.013,0.911,0.000,NaN
1,Form,3065.866,1,608,3065.866,131.830,0.000,0.047,1.0
2,Interaction,36.349,1,608,36.349,1.563,0.212,0.001,NaN
